In [1]:
## import nba_api to collect information
from nba_api.stats import endpoints
from nba_api.stats.static import teams 
teams = teams.get_teams()

## other pacakges used
import pandas as pd
import numpy as np

In [2]:
##team_list = ['1610612747','1610612738','1610612741']
team_list = ['1610612747','1610612738','1610612741','1610612762','1610612746','1610612743','1610612756','1610612755','1610612737','1610612749','1610612751','1610612739','1610612740','1610612744','1610612752']

In [3]:
team_data = pd.DataFrame()

for x in team_list:
    data = endpoints.teamyearbyyearstats.TeamYearByYearStats(team_id=x)
    df = data.get_data_frames()[0]
    
    if len(team_data.columns) == 0:
        team_data = df
    else:
        team_data = team_data.append(df)

In [4]:
#only use selected columns from data set
team_data = team_data[['TEAM_ID','TEAM_NAME','NBA_FINALS_APPEARANCE','FGM','FGA','FG_PCT','FG3M','FG3A','FG3_PCT','FTM','FTA','FT_PCT','OREB','DREB','REB','AST','PF','STL','TOV','BLK']]

#clean data set so that only data is used when FGA information is captured
team_data = team_data[team_data.FGA > 0]

#create column taht indicates in team won championship or not. 1 = champions
team_data['CHAMPION'] = team_data['NBA_FINALS_APPEARANCE'].apply(lambda x: 1 if x == 'LEAGUE CHAMPION' else 0)

#drop NBA_FINALS_APPEARANCE column as not used
team_data.drop(columns=['NBA_FINALS_APPEARANCE'], inplace=True)

In [5]:
team_data.head()

,TEAM_ID,TEAM_NAME,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,PF,STL,TOV,BLK,CHAMPION
34,1610612747,Lakers,3964,7512,0.528,10,96,0.104,1495,2031,0.736,1235,2433,3668,2519,1931,844,1584,479,0
35,1610612747,Lakers,3854,7250,0.532,58,226,0.257,1712,2272,0.754,1095,2499,3594,2455,2054,726,1578,478,0
36,1610612747,Lakers,3952,7254,0.545,90,295,0.305,1702,2232,0.763,1063,2550,3613,2575,1931,695,1537,481,1
37,1610612747,Lakers,3834,7343,0.522,138,409,0.337,1812,2329,0.778,1101,2555,3656,2433,2031,693,1467,419,0
38,1610612747,Lakers,3740,7245,0.516,164,447,0.367,2012,2550,0.789,1127,2515,3642,2428,1853,728,1358,482,1


In [6]:
import pandas as pd
from pathlib import Path
from sklearn import tree
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
import pydotplus
from IPython.display import Image
from sklearn.preprocessing import LabelEncoder
from imblearn.ensemble import BalancedRandomForestClassifier

In [7]:
le = LabelEncoder()

In [8]:
le.fit(team_data["TEAM_NAME"])
team_data["TEAM_NAME"] = le.transform(team_data["TEAM_NAME"])
team_data.drop(columns=['TEAM_NAME'], inplace=True)

In [9]:
team_data.head()

,TEAM_ID,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,PF,STL,TOV,BLK,CHAMPION
34,1610612747,3964,7512,0.528,10,96,0.104,1495,2031,0.736,1235,2433,3668,2519,1931,844,1584,479,0
35,1610612747,3854,7250,0.532,58,226,0.257,1712,2272,0.754,1095,2499,3594,2455,2054,726,1578,478,0
36,1610612747,3952,7254,0.545,90,295,0.305,1702,2232,0.763,1063,2550,3613,2575,1931,695,1537,481,1
37,1610612747,3834,7343,0.522,138,409,0.337,1812,2329,0.778,1101,2555,3656,2433,2031,693,1467,419,0
38,1610612747,3740,7245,0.516,164,447,0.367,2012,2550,0.789,1127,2515,3642,2428,1853,728,1358,482,1


In [10]:
target = ["CHAMPION"]
y = team_data.loc[:, target].copy()

X = team_data.drop(columns="CHAMPION")
X = pd.get_dummies(X)

In [11]:
X.describe()

,TEAM_ID,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,PF,STL,TOV,BLK
count,5.650000e+02,565.000000,565.000000,565.000000,565.000000,565.000000,565.000000,565.000000,565.000000,565.000000,565.000000,565.000000,565.000000,565.000000,565.000000,565.000000,565.000000,565.000000
mean,1.610613e+09,3116.952212,6738.700885,0.461827,432.419469,1229.835398,0.336582,1560.389381,2066.201770,0.755265,987.263717,2447.070796,3434.334513,1872.637168,1784.782301,651.865487,1243.759292,409.208850
std,7.180999e+00,367.651102,635.305671,0.022656,265.650225,707.110946,0.042004,256.801391,330.683863,0.028819,180.590431,248.942477,305.292952,264.199449,238.764032,99.980847,170.004762,86.607058
min,1.610613e+09,1539.000000,3561.000000,0.401000,10.000000,96.000000,0.104000,745.000000,1077.000000,0.664000,472.000000,1293.000000,1788.000000,782.000000,987.000000,346.000000,681.000000,169.000000
25%,1.610613e+09,2955.000000,6537.000000,0.445000,224.000000,668.000000,0.323000,1402.000000,1858.000000,0.738000,867.000000,2360.000000,3340.000000,1716.000000,1663.000000,585.000000,1149.000000,350.000000
50%,1.610613e+09,3121.000000,6804.000000,0.461000,415.000000,1187.000000,0.346000,1549.000000,2085.000000,0.756000,988.000000,2457.000000,3474.000000,1860.000000,1797.000000,648.000000,1238.000000,400.000000
75%,1.610613e+09,3349.000000,7094.000000,0.476000,602.000000,1666.000000,0.363000,1736.000000,2300.000000,0.775000,1105.000000,2559.000000,3612.000000,2049.000000,1937.000000,718.000000,1344.000000,465.000000
max,1.610613e+09,3964.000000,8868.000000,0.545000,1205.000000,3134.000000,0.416000,2316.000000,2883.000000,0.839000,1520.000000,3316.000000,4078.000000,2575.000000,2398.000000,987.000000,1873.000000,697.000000


In [12]:
y['CHAMPION'].value_counts()

0    542
1     23
Name: CHAMPION, dtype: int64

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [14]:
scaler = StandardScaler()

In [15]:
X_scaler = scaler.fit(X_train)

In [16]:
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [17]:
brf = BalancedRandomForestClassifier(n_estimators=100, random_state=1)
brf.fit(X_train_scaled, y_train)

<ipython-input-17-03a5d3e36309>:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  brf.fit(X_train_scaled, y_train)


BalancedRandomForestClassifier(random_state=1)

In [18]:
y_pred = brf.predict(X_test_scaled)
balanced_accuracy_score(y_test, y_pred)

NameError: name 'balanced_accuracy_score' is not defined

In [19]:
confusion_matrix(y_test, y_pred)

array([[104,  30],
       [  3,   5]], dtype=int64)

In [20]:
print(classification_report_imbalanced(y_test, y_pred))

NameError: name 'classification_report_imbalanced' is not defined

In [21]:
importances = brf.feature_importances_ 
indices = np.argsort(importances)[::-1]
for f in range(X.shape[1]):
    print(f"{X.columns.values[indices[f]]}: ({importances[indices[f]]})")

FG_PCT: (0.1336937074442559)
REB: (0.10422277468170683)
TOV: (0.08418667611007954)
TEAM_ID: (0.0730791667312381)
FGM: (0.07210091691728739)
FG3_PCT: (0.06789229551452824)
PF: (0.05963114224345797)
AST: (0.05801002862080446)
FTM: (0.05724597497539829)
FTA: (0.055132677430255086)
DREB: (0.04099182398450469)
FG3A: (0.03749299054800632)
FGA: (0.03685693093340997)
STL: (0.03557763241536826)
BLK: (0.028317500831443517)
FT_PCT: (0.021844827438989904)
FG3M: (0.020561557588514204)
OREB: (0.01316137559075139)


In [30]:
### Select team in playoffs to predict winner

##Jazz - 1610612762, Clippers - 1610612746, Denver - 1610612743, Pheonix - 1610612756, 76ers - 1610612755 , Hawks - 1610612737, Bucks - 1610612749, Nets - 1610612751
teams_playoffs = ['1610612762','1610612746','1610612743','1610612756','1610612755','1610612737','1610612749','1610612751']
##teams_playoffs = ['1610612751']
teams_playoffs

['1610612762',
 '1610612746',
 '1610612743',
 '1610612756',
 '1610612755',
 '1610612737',
 '1610612749',
 '1610612751']

In [31]:
from nba_api.stats import endpoints
team_data_playoff = pd.DataFrame()

for x in teams_playoffs:
    data = endpoints.teamyearbyyearstats.TeamYearByYearStats(team_id=x)
    df = data.get_data_frames()[0]
    
    if len(team_data_playoff.columns) == 0:
        team_data_playoff = df
    else:
        team_data_playoff = team_data_playoff.append(df)

In [32]:
team_data_playoff

,TEAM_ID,TEAM_CITY,TEAM_NAME,YEAR,GP,WINS,LOSSES,WIN_PCT,CONF_RANK,DIV_RANK,...,OREB,DREB,REB,AST,PF,STL,TOV,BLK,PTS,PTS_RANK
0,1610612762,New Orleans,Jazz,1974-75,82,23,59,0.280,9,5,...,1144,2616,3760,1818,2222,725,1802,256,8319,10
1,1610612762,New Orleans,Jazz,1975-76,82,38,44,0.463,7,4,...,1189,2779,3968,1765,2175,750,1659,343,8535,10
2,1610612762,New Orleans,Jazz,1976-77,82,35,47,0.427,8,5,...,1249,2828,4077,1854,2099,613,1706,357,8574,16
3,1610612762,New Orleans,Jazz,1977-78,82,39,43,0.476,7,5,...,1309,2907,4216,2079,1938,662,1694,514,8826,13
4,1610612762,New Orleans,Jazz,1978-79,82,26,56,0.317,11,6,...,1234,2676,3910,2079,1940,760,1764,559,8882,15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40,1610612751,Brooklyn,Nets,2016-17,82,20,62,0.244,15,5,...,721,2879,3600,1756,1726,593,1356,387,8673,12
41,1610612751,Brooklyn,Nets,2017-18,82,28,54,0.341,12,5,...,792,2852,3644,1941,1688,512,1245,390,8741,14
42,1610612751,Brooklyn,Nets,2018-19,82,42,40,0.512,6,4,...,900,2919,3819,1954,1763,539,1236,339,9204,15
43,1610612751,Brooklyn,Nets,2019-20,72,35,37,0.486,7,4,...,766,2683,3449,1761,1510,464,1099,323,8048,17


In [33]:
#select this season only
team_data_playoff = team_data_playoff[team_data_playoff.YEAR == '2020-21']

#only use selected columns from data set
team_data_playoff = team_data_playoff[['TEAM_ID','TEAM_NAME','NBA_FINALS_APPEARANCE','FGM','FGA','FG_PCT','FG3M','FG3A','FG3_PCT','FTM','FTA','FT_PCT','OREB','DREB','REB','AST','PF','STL','TOV','BLK']]

#clean data set so that only data is used when FGA information is captured
team_data_playoff = team_data_playoff[team_data_playoff.FGA > 0]

#create column taht indicates in team won championship or not. 1 = champions
team_data_playoff['CHAMPION'] = team_data_playoff['NBA_FINALS_APPEARANCE'].apply(lambda x: 1 if x == 'LEAGUE CHAMPION' else 0)

#drop NBA_FINALS_APPEARANCE column as not used
team_data_playoff.drop(columns=['NBA_FINALS_APPEARANCE'], inplace=True)
team_data_playoff.drop(columns=['CHAMPION'], inplace=True)
team_data_playoff_final = team_data_playoff.copy()
team_data_playoff.drop(columns=['TEAM_NAME'], inplace=True)

In [34]:
y_pred_winner = brf.predict(team_data_playoff)

In [35]:
y_pred_winner

array([0, 0, 0, 0, 0, 0, 0, 0], dtype=int64)

In [36]:
team_data_playoff_final.insert(0, 'CHAMPION?', y_pred_winner, allow_duplicates=False)

In [37]:
team_data_playoff_final

,CHAMPION?,TEAM_ID,TEAM_NAME,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,PF,STL,TOV,BLK
46,0,1610612762,Jazz,2971,6344,0.468,1205,3098,0.389,1235,1546,0.799,765,2709,3474,1703,1335,474,1023,371
50,0,1610612746,Clippers,3009,6242,0.482,1027,2498,0.411,1164,1387,0.839,678,2501,3179,1756,1383,509,949,295
44,0,1610612743,Nuggets,3114,6422,0.485,927,2462,0.377,1129,1406,0.803,758,2442,3200,1933,1374,582,972,323
52,0,1610612756,Suns,3118,6357,0.490,940,2490,0.378,1124,1347,0.834,630,2462,3092,1939,1374,517,902,312
71,0,1610612755,76ers,2981,6257,0.476,811,2169,0.374,1409,1836,0.767,722,2522,3244,1706,1456,655,1040,447
71,0,1610612737,Hawks,2937,6281,0.468,895,2402,0.373,1417,1745,0.812,760,2525,3285,1737,1392,503,953,342
52,0,1610612749,Bucks,3221,6610,0.487,1038,2669,0.389,1169,1539,0.760,741,2724,3465,1834,1244,585,995,334
44,0,1610612751,Nets,3106,6289,0.494,1020,2600,0.392,1305,1623,0.804,640,2559,3199,1929,1371,484,975,379
